**Open text file with song lyrics**

In [3]:
with open('main_df_ns.txt', 'r') as f:
  file = f.read()

**then, using Natasha package, make a lemmatizated text**

In [1]:
!pip install natasha

In [14]:
from natasha import (
    Segmenter,
    MorphVocab,

    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,

    PER,
    NamesExtractor,

    Doc
)


segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

text = file
doc = Doc(text)
doc.segment(segmenter)
doc.tag_morph(morph_tagger)
doc.tag_ner(ner_tagger)
for token in doc.tokens:
    token.lemmatize(morph_vocab)
lemman_text = [_.lemma for _ in doc.tokens]
lemman_text = ' '.join(lemman_text)

**Next make a bigrams from text using nltk without punctuation marks**

In [8]:
import nltk
from nltk.util import ngrams
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import re

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/artemrozkov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
def extract_ngrams(data, num):
    n_grams = ngrams(nltk.word_tokenize(data), num)
    return [ ' '.join(grams) for grams in n_grams]
def bi_grams(text, n = 2):
  result = re.sub(r'[^\w\s]+', ' ', text)
  return extract_ngrams(result, n)

['fgergerfwe fefe', 'fefe r', 'r ewfef']

In [15]:
ns_bi_grams = bi_grams(lemman_text, 2)

**Filter pairs with more that 3-character word**

In [17]:
filtered_word_combinations = []

# Проходим по каждому словосочетанию и проверяем слова в нем
for combination in ns_bi_grams:
    words = combination.split()  # Разделяем словосочетание на отдельные слова
    short_words = [word for word in words if len(word) <= 3]  # Выбираем слова с менее чем 3 символами
    if len(short_words) == 0:
        filtered_word_combinations.append(combination)  # Если нет таких слов, добавляем словосочетание в новый список


In [21]:
print(' ns_bi_grams len: ', len(ns_bi_grams), '\n', 'filtered_word_combinations len: ', len(filtered_word_combinations))
#compare a lists size

 ns_bi_grams len:  2166 
 filtered_word_combinations len:  726


**Transfer list to turple for network**

In [18]:
edges = [tuple(item.split()) for item in filtered_word_combinations]
edges

[('большой', 'широкий'),
 ('широкий', 'город'),
 ('город', 'магистраль'),
 ('окно', 'бесполезный'),
 ('бесполезный', 'тюрьма'),
 ('тюрьма', 'зеленый'),
 ('зеленый', 'яблоко'),
 ('яблоко', 'железо'),
 ('железо', 'запеть'),
 ('стать', 'слаще'),
 ('японский', 'лагерь'),
 ('лагерь', 'пропасть'),
 ('пропасть', 'голубь'),
 ('голубь', 'синица'),
 ('проводить', 'ночь'),
 ('ночь', 'тридцать'),
 ('тридцать', 'первый'),
 ('первый', 'весна'),
 ('сомнение', 'ревновать'),
 ('сомнение', 'ревновать'),
 ('весь', 'бить'),
 ('бить', 'стекло'),
 ('город', 'просолить'),
 ('просолить', 'куда'),
 ('любой', 'время'),
 ('доходить', 'поезд'),
 ('губа', 'часто'),
 ('часто', 'здесь'),
 ('здесь', 'обветрить'),
 ('бывать', 'рассвет'),
 ('соломе', 'крыло'),
 ('крыло', 'паутина'),
 ('паутина', 'провод'),
 ('мягкий', 'тигр'),
 ('тигр', 'сторож'),
 ('нежность', 'вода'),
 ('вода', 'несовершенный'),
 ('этот', 'март'),
 ('этот', 'март'),
 ('март', 'навсегда'),
 ('одуреть', 'дрейфовать'),
 ('дрейфовать', 'корабль'),
 ('кор

**Calculate a node weights, based on word frequency**

In [22]:
word_weights = {}

# Перебираем каждое уникальное словосочетание
for phrase in filtered_word_combinations:
    # Разбиваем словосочетание на отдельные слова
    words = phrase.split()

    # Для каждого слова увеличиваем его вес в словаре
    for word in words:
        if word in word_weights:
            word_weights[word] += 1
        else:
            word_weights[word] = 1

print(word_weights)


{'большой': 1, 'широкий': 2, 'город': 8, 'магистраль': 1, 'окно': 3, 'бесполезный': 2, 'тюрьма': 2, 'зеленый': 2, 'яблоко': 2, 'железо': 2, 'запеть': 1, 'стать': 15, 'слаще': 1, 'японский': 1, 'лагерь': 2, 'пропасть': 4, 'голубь': 2, 'синица': 1, 'проводить': 1, 'ночь': 14, 'тридцать': 2, 'первый': 2, 'весна': 2, 'сомнение': 2, 'ревновать': 2, 'весь': 34, 'бить': 2, 'стекло': 1, 'просолить': 2, 'куда': 1, 'любой': 1, 'время': 9, 'доходить': 1, 'поезд': 1, 'губа': 7, 'часто': 2, 'здесь': 4, 'обветрить': 1, 'бывать': 1, 'рассвет': 5, 'соломе': 1, 'крыло': 4, 'паутина': 2, 'провод': 1, 'мягкий': 1, 'тигр': 4, 'сторож': 1, 'нежность': 1, 'вода': 5, 'несовершенный': 1, 'этот': 7, 'март': 3, 'навсегда': 2, 'одуреть': 1, 'дрейфовать': 2, 'корабль': 5, 'неблагодарный': 1, 'любить': 9, 'цветок': 1, 'книга': 1, 'портвейн': 1, 'театр': 1, 'боно': 1, 'бритни': 1, 'термос': 1, 'карликовый': 1, 'рост': 2, 'молоко': 1, 'брют': 1, 'joint': 1, 'нибудь': 2, 'подарить': 3, 'зоопарк': 1, 'джунгли': 1, 'да